## Track with TOBAC/TINT instead of standalone TINT

In [1]:
import numpy as np

In [2]:
import tobac
from tobac.themes import tint

In [3]:
import matplotlib.pyplot as plt
%matplotlib widget

import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
from mpl_toolkits.axes_grid1 import AxesGrid

In [4]:
nc_file_path = ('./Tgrid_0*.nc')

In [5]:
from copy import deepcopy
track_params = deepcopy(tint.objects.default_params)
track_params

{'FIELD_THRESH': 32,
 'ISO_THRESH': 8,
 'ISO_SMOOTH': 3,
 'MIN_SIZE': 8,
 'SEARCH_MARGIN': 4000,
 'FLOW_MARGIN': 10000,
 'MAX_DISPARITY': 999,
 'MAX_FLOW_MAG': 50,
 'MAX_SHIFT_DISP': 15,
 'GS_ALT': 1500}

In [6]:
track_params['GS_ALT'] = 3000
track_params['FIELD_THRESH'] = 30

In [7]:
# jsparams={'FIELD_THRESH': 30,
#  'MIN_SIZE': 8,
#  'SEARCH_MARGIN': 4000,
#  'FLOW_MARGIN': 10000,
#  'MAX_FLOW_MAG': 50,
#  'MAX_DISPARITY': 999,
#  'MAX_SHIFT_DISP': 15,
#  'ISO_THRESH': 8,
#  'ISO_SMOOTH': 3,
#  'GS_ALT': 3000}
# for k in jsparams:
#     print(jsparams[k] == track_params[k])

In [8]:
nc_grid = tint.io.load_cfradial_grids(nc_file_path)
# print(nc_grid)
tracks = tint.make_tracks(nc_grid, 'reflectivity', params=track_params)
print(tracks)

/Users/ebruning/miniconda3/envs/tobac-dev/lib/python3.9/site-packages/xarray/core/dataarray.py:745: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return key in self.data


Writing tracks for scan 0
Writing tracks for scan 1
Writing tracks for scan 2
Writing tracks for scan 3
Writing tracks for scan 4
Writing tracks for scan 5
Writing tracks for scan 6
Writing tracks for scan 7
Writing tracks for scan 8
Writing tracks for scan 9
Writing tracks for scan 10
Writing tracks for scan 11
Writing tracks for scan 12
Writing tracks for scan 13
Writing tracks for scan 14
Writing tracks for scan 15
Writing tracks for scan 16
Writing tracks for scan 17
Writing tracks for scan 18
Writing tracks for scan 19
Writing tracks for scan 20
Writing tracks for scan 21
Writing tracks for scan 22
Writing tracks for scan 23
Writing tracks for scan 24
Writing tracks for scan 25
Writing tracks for scan 26
Writing tracks for scan 27
Writing tracks for scan 28
Writing tracks for scan 29
Writing tracks for scan 30
Writing tracks for scan 31
Writing tracks for scan 32
Writing tracks for scan 33
Writing tracks for scan 34
Writing tracks for scan 35
Writing tracks for scan 36
Writing tra

In [9]:
nc_grid

<xarray.Dataset>
Dimensions:                     (nradar: 1, storm: 2, time: 100, x: 501, y: 501, z: 31)
Coordinates:
  * x                           (x) float64 -2.5e+05 -2.49e+05 ... 2.5e+05
  * y                           (y) float64 -2.5e+05 -2.49e+05 ... 2.5e+05
  * z                           (z) float64 0.0 500.0 1e+03 ... 1.45e+04 1.5e+04
  * time                        (time) datetime64[ns] 2017-07-13T08:00:47.333...
Dimensions without coordinates: nradar, storm
Data variables: (12/22)
    origin_latitude             (time) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    origin_longitude            (time) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    origin_altitude             (time) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    projection                  (time) int32 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    ProjectionCoordinateSystem  (time) int32 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    radar_latitude              (time, nradar) float64 dask.array<chunksize=(1, 1), meta=np.ndarray>
    ...                          ...
    MW                          (time, z, y, x) float32 dask.array<chunksize=(1, 31, 501, 501), meta=np.ndarray>
    MI                          (time, z, y, x) float32 dask.array<chunksize=(1, 31, 501, 501), meta=np.ndarray>
    ROI                         (time, z, y, x) float32 dask.array<chunksize=(1, 31, 501, 501), meta=np.ndarray>
    point_latitude              (z, y, x) float64 27.2 27.2 27.2 ... 31.69 31.69
    point_longitude             (z, y, x) float64 -97.61 -97.6 ... -92.45 -92.44
    storm_id                    (storm) int64 0 1
Attributes: (12/18)
    Conventions:         CF/Radial instrument_parameters
    version:             1.3
    title:               
    institution:         
    references:          
    source:              
    ...                  ...
    volume_number:       0
    platform_type:       fixed
    instrument_type:     radar
    primary_axis:        axis_z
    cf_tree_order:       storm_id cell_id
    tree_id:             45896

In [10]:
# tracks.cell_mask

In [11]:
# tracks

In [12]:
# Copy over coordinate data, and fix swapped cell_mask coordinates.
tracks2=tracks.swap_dims({'x':'y2', 'y':'x2'}).rename_dims({'x2':'x', 'y2':'y'})
tracks2['x']=nc_grid['x']
tracks2['y']=nc_grid['y']
tracks2['z']=nc_grid['z']

# Copy the coordinate center for the gridded radar data
# tracks2['origin_latitude'] = nc_grid.origin_latitude
# tracks2['origin_longitude'] = nc_grid.origin_longitude

# tracks2['ProjectionCoordinateSystem'] = nc_grid.ProjectionCoordinateSystem

# Copy data variables
# for var in nc_grid.variables.keys():
#     if nc_grid[var].dims==('time', 'z', 'y', 'x'):
#         tracks2[var] = nc_grid[var]
# tracks2

# Also, grid_x and grid_y are grid box indexes, not coordinates. Is that what the spec indicates, or is the spec ambiguous?

In [13]:
# import warnings
# warnings.filterwarnings('ignore')

# from ipywidgets import interact, widgets
# import pyart

# plt.close('all')
# fig = plt.figure(figsize=(11,8.5))
# all_t = tracks2.time.data

# cell_max_id = tracks2.cell_id.astype('uint32').data.max()

# @interact(t=widgets.SelectionSlider(options={v:i for i, v in enumerate(all_t)}))
# def quick_cell_mask_plot(t):
#     fig.clear()
#     axs = fig.subplots(2,2, sharex=True, sharey=True)
#     print(t)
#     scan = {'time':t}
    
#     data = tracks2.cell_mask[scan]
#     print(data)
#     data.where(data!=0, np.nan).plot.imshow(ax=axs[0,0], vmin=0, vmax=cell_max_id, cmap='prism')

#     data = nc_grid.reflectivity[scan].max(axis=0) # Column max 
#     data.where(data!=0, np.nan).plot.imshow(ax=axs[0,1], vmin=-24, vmax=72, cmap='pyart_LangRainbow12')

#     data = nc_grid.differential_reflectivity[scan].max(axis=0) # Column max 
#     data.where(data!=0, np.nan).plot.imshow(ax=axs[1,1], vmin=-2, vmax=6, cmap='pyart_LangRainbow12')

#     data = nc_grid.KDP_CSU[scan].max(axis=0) # Column max 
#     data.where(data!=0, np.nan).plot.imshow(ax=axs[1,0], vmin=-2, vmax=3, cmap='pyart_LangRainbow12')

#     # 8 UTC isolated cell in Jessica's openradar/TINT notebooks.
#     # Need to subtract 250 km to shift to the grid box coordinates used by TINT.
# #     axs[0,0].set_xlim(145e3-250e3, 205e3-250e3)
# #     axs[0,0].set_ylim(165e3-250e3, 225e3-250e3)    
    
#     # 20 UTC
#     axs[0,0].set_xlim(0, 250e3)
#     axs[0,0].set_ylim(0, 250e3)

In [14]:
import xarray as xr
import pandas as pd

In [15]:
class TrackSummary(object):
    def __init__(self, grid_ds, tracks_ds):
        """ Convenience API for pulling data out of a TOBAC/TINT tracks object """
        
        # Should push this change upstream into TOBAC/TINT
        self.grid_ds = grid_ds
        self.tracks_ds = tracks_ds.drop('cell_id').assign_coords(cell=tracks_ds['cell'].astype('uint64'))
        track_table = self.tracks_ds.drop_dims([d for d in self.tracks_ds.dims if d != 'cell'])
        self.track_groups = track_table.groupby('cell')
#         self.most_freq = self.track_groups.size().sort_values(ascending=False).index
#         most_freq would be found by sorting cell_scan_count
        cell_ids = {cid:len(v) for cid, v in self.track_groups.groups.items()}
        id_data = np.fromiter(cell_ids.keys(), dtype=int)
        count_data = np.fromiter(cell_ids.values(), dtype=int)
        cell_scan_count = xr.DataArray(count_data, coords={'cell':id_data}, dims=tracks2.cell_id.dims, name='cell_scan_count')
        self.all_times = np.sort(np.unique(self.tracks_ds.cell_time))
        
        # Calculate some stats
        # min of always, sometimes, never isolated
        # min(True, True, True) = True
        # min(True, False, True) = False
        # min(False, False, False) = False
        # max of always, sometimes, never isolated
        # max(True, True, True) = True
        # max(True, False, True) = True
        # max(False, False, False) = False        
        
        # Assume proj is aeqd, the default for PyART.
        cs_attrs = self.grid_ds['ProjectionCoordinateSystem'][0].attrs
#         {'latitude_of_projection_origin': 29.471900939941406,
#          'longitude_of_projection_origin': -95.0787353515625,
#          '_CoordinateTransformType': 'Projection',
#          '_CoordinateAxes': 'x y z time',
#          '_CoordinateAxesTypes': 'GeoX GeoY Height Time',
#          'grid_mapping_name': 'azimuthal_equidistant',
#          'semi_major_axis': 6370997.0,
#          'inverse_flattening': 298.25,
#          'longitude_of_prime_meridian': 0.0,
#          'false_easting': 0.0,
#          'false_northing': 0.0}
        if cs_attrs['grid_mapping_name'] == 'azimuthal_equidistant':
            self.grid_proj = ccrs.AzimuthalEquidistant(central_latitude=cs_attrs['latitude_of_projection_origin'],
                                                       central_longitude=cs_attrs['longitude_of_projection_origin'],
                                                       false_easting=cs_attrs['false_easting'],
                                                       false_northing=cs_attrs['false_northing'],)
        
        column_names = list(track_table.variables.keys())
        column_names.remove('cell')
#         column_names.remove('cell_id')
        track_mins = self.track_groups.min().rename({n:n+'_min' for n in column_names})
        track_maxs = self.track_groups.max().rename({n:n+'_max' for n in column_names})
                
        track_stats = xr.merge([track_mins, track_maxs])
        track_stats['scan_count'] = cell_scan_count
        
        sum_vars = ['isolated'] 
        sum_ds = track_table.drop([c for c in column_names if c not in sum_vars])
        track_sums = sum_ds.groupby('cell').sum()
        track_stats['isolated_count'] = track_sums.isolated
        track_stats['isolated_fraction'] = track_stats['isolated_count']/track_stats['scan_count']
        self.track_stats = track_stats
                
    def active_track_ids(self, time):
        t = pd.Timestamp(time)
        in_range = ((t >= self.track_stats['cell_time_min']) & 
                    (t < self.track_stats['cell_time_max'])
                   )
        return self.track_stats.cell[{'cell':in_range}]
    
    def active_grid(self, time, field):
        t = time
        if field == 'cell_mask':
            return self.tracks_ds[field].sel({'time':t}, method='nearest')
        else:
            return self.grid_ds[field].sel({'time':t}, method='nearest')
                
    def active_track_stats(self, time):
        """ Get the stats dataframe for active tracks at time"""
        uids = self.active_track_ids(time)
        return self.track_stats.loc[{'cell':uids}]
    
    def widget(self, isolated_increment=0.05):
        from ipywidgets import interact, widgets
        from functools import partial
        time_selection_mapping = {v:i for i, v in enumerate(self.all_times)}
        slider_layout = widgets.Layout(width='50%')
        time_slider = widgets.SelectionSlider(options=time_selection_mapping, 
                                              layout=slider_layout, description='Time')
        isolated_min_slider = widgets.FloatSlider(min=0.0, max=1.0, step=isolated_increment, 
                                                  layout=slider_layout, description="Isolation")
        show_tracks = widgets.Checkbox(value=True, description='Tracks')

        return partial(interact,
                       time=time_slider,
                       isolated_min=isolated_min_slider,
                       show_tracks = show_tracks
                      )

In [16]:
summary = TrackSummary(nc_grid,tracks2)

In [27]:
import pyart # To get colormaps

plt.close('all')
fig = plt.figure(figsize=(9,9))

hou_ctr_lat, hou_ctr_lon = 29.4719, -95.0792

@summary.widget()
def plot(time, isolated_min, show_tracks):
    # Get the data
    t = summary.all_times[time] # convert slider index to time
    stats = summary.active_track_stats(t)
    isolated_enough = (stats['isolated_fraction'] >= isolated_min)
    isolated_track_ids = stats[{'cell':isolated_enough}].cell
    
    all_cell_ids = np.unique(summary.track_stats.cell)
    cell_min_id = all_cell_ids.min()
    cell_max_id = all_cell_ids.max()
        
    # Plot
    fig.clear()
    latlon_proj = ccrs.PlateCarree()
    projection=summary.grid_proj
    axes_class = (GeoAxes,
                  dict(map_projection=projection))
    axs = AxesGrid(fig, 111, axes_class=axes_class,
                nrows_ncols=(2, 2),
                axes_pad=0.4,
                cbar_location='right',
                cbar_mode='each',
                cbar_pad=0.4,
                cbar_size='3%',
                label_mode='')  # note the empty label_mode
    for ax in axs:
        ax.coastlines()
        ax.set_extent((hou_ctr_lon-1.0, hou_ctr_lon+1.0,
                       hou_ctr_lat-1.0, hou_ctr_lat+1.0))
    
    
    # Gridded background
    grid_extent = (summary.grid_ds.x.min(), summary.grid_ds.x.max(),
                   summary.grid_ds.y.min(), summary.grid_ds.y.max())

    
    # Cell ID
    cell_id = summary.active_grid(t, 'cell_mask')
    cell_id = cell_id.where(cell_id!=0, np.nan)
    im = axs[0].imshow(cell_id, vmin=0, vmax=cell_max_id, cmap='prism',
                  transform=summary.grid_proj, extent=grid_extent, origin='lower')
    axs[0].set_title('Cell ID')
    axs.cbar_axes[0].colorbar(im)

    # Column max reflectivity
    refl = summary.active_grid(t, 'reflectivity').max(axis=0)
#     data = nc_grid.reflectivity[{'time':t}].max(axis=0).where(data!=0, np.nan) # Column max 
    im = axs[1].imshow(refl, vmin=-24, vmax=72, cmap='pyart_LangRainbow12',
                  transform=summary.grid_proj, extent=grid_extent, origin='lower')
    axs[1].set_title('Reflectivity (dBZ)')
    axs.cbar_axes[1].colorbar(im)
    
    # Column max Zdr
    refl = summary.active_grid(t, 'differential_reflectivity').max(axis=0)
#     data = nc_grid.reflectivity[{'time':t}].max(axis=0).where(data!=0, np.nan) # Column max 
    im = axs[2].imshow(refl, vmin=-2, vmax=6, cmap='pyart_LangRainbow12',
                  transform=summary.grid_proj, extent=grid_extent, origin='lower')
    axs[2].set_title('Differential Reflectivity (dB)')
    axs.cbar_axes[2].colorbar(im)
    
    # Column max Kdp
    refl = summary.active_grid(t, 'KDP_CSU').max(axis=0)
#     data = nc_grid.reflectivity[{'time':t}].max(axis=0).where(data!=0, np.nan) # Column max 
    im = axs[3].imshow(refl, vmin=-2, vmax=3, cmap='pyart_LangRainbow12',
                  transform=summary.grid_proj, extent=grid_extent, origin='lower')
    axs[3].set_title('Specific Differential Phase (deg/km)')
    axs.cbar_axes[3].colorbar(im)

    
    have_data = False
    if show_tracks == True:
        for tid in isolated_track_ids:
            # loop over dataframes for each cell
            track = summary.track_groups[int(tid)]
            t_stat = stats.loc[{'cell':tid}]        
            time_offsets = track['cell_time'] - t
            cell_idx = int(abs(time_offsets).argmin())
            cell_now = track[{'cell':cell_idx}]

            for axi in range(len(axs)):
                axs[axi].plot(track['longitude'], track['latitude'], transform=latlon_proj,
                              color='black', alpha=0.5, linewidth=1.0)

                cell_lon, cell_lat = cell_now['longitude'], cell_now['latitude']
                last_scatter = axs[axi].scatter(cell_lon, cell_lat, c=[float(tid)], s=9, marker='s', cmap='prism', 
                               vmin=cell_min_id, vmax=cell_max_id, transform=latlon_proj)
                # work around text not being clipped properly by matplotlib (known issue)
        #         transformed_center = axs[0].background_patch.get_transform().transform((cell_lon, cell_lat))
        #         print(transformed_center)
        #         print(axs[0].bbox)
        #         if axs[0].background_patch.contains_point(transformed_center):
                txt_art = axs[axi].text(cell_now['longitude'], cell_now['latitude'], int(tid), transform=latlon_proj,
                                       fontsize=9)
            have_data = True

#     if have_data:
#         axs.cbar_axes[0].colorbar(last_scatter) #only need last scatter since they all have the same cmap

/Users/ebruning/miniconda3/envs/tobac-dev/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/ebruning/miniconda3/envs/tobac-dev/lib/python3.9/site-packages/ipywidgets/widgets/widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Toolbar).__init__().
__init__() missing 1 required positional argument: 'canvas'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/Users/ebruning/miniconda3/envs/tobac-dev/lib/python3.9/site-packages/ipywidgets/widgets/widget_selection.py:121: DeprecationWarning: Support for mapping types has been deprecated and will be dropped in a future release.
  warnings.warn("Support for mapping types has been deprecated and will be dropped in a future release.", DeprecationWarning)
/Users/ebruning/miniconda3/envs/tobac-dev/lib/python3.9/site-packages/ipywidgets/widgets/widget_selection.py:121: DeprecationWarning: Support for mapping types has been deprecated and will be dropped in a future release.
  warnings.warn("Support for mapping types has been deprecated and will be dropped in a future release.", DeprecationWarning)


interactive(children=(SelectionSlider(description='Time', layout=Layout(width='50%'), options={numpy.datetime6…